# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
nltk.download('stopwords')

from sqlalchemy import create_engine
import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to /Users/jimeng/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/jimeng/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jimeng/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jimeng/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('Disaster', con=engine)
X = df.message
y = df[df.columns[4:]]

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).strip()
        clean_tokens.append(clean_tok)
    
    clean_tokens = [w for w in clean_tokens if w not in stopwords.words("english")]

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer()),
            ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [8]:
#use 5000 points as training samples to tune inital parameters faster
X_sample_train = X_train[:5000]
y_sample_train = y_train[:5000]

In [9]:
pipeline.fit(X_sample_train, y_sample_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

In [10]:
y_pred = pipeline.predict(X_test)


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
accuracy = (y_pred == y_test).mean()

print("Accuracy:\n",accuracy)

Accuracy:
 related                   0.805417
request                   0.888974
offer                     0.995390
aid_related               0.771418
medical_help              0.923934
medical_products          0.952939
search_and_rescue         0.973300
security                  0.980023
military                  0.966193
child_alone               1.000000
water                     0.951786
food                      0.938532
shelter                   0.934883
clothing                  0.984633
money                     0.976566
missing_people            0.992317
refugees                  0.966193
death                     0.959854
other_aid                 0.866500
infrastructure_related    0.932962
transport                 0.953899
buildings                 0.951018
electricity               0.978871
tools                     0.992509
hospitals                 0.988283
shops                     0.994622
aid_centers               0.989243
other_infrastructure      0.954860
weather_r

In [12]:
print(classification_report(y_test.values, y_pred, target_names=y.keys()))

                        precision    recall  f1-score   support

               related       0.82      0.95      0.88      3992
               request       0.81      0.48      0.60       913
                 offer       0.00      0.00      0.00        24
           aid_related       0.76      0.66      0.70      2161
          medical_help       0.62      0.04      0.08       403
      medical_products       0.75      0.04      0.07       251
     search_and_rescue       0.75      0.02      0.04       141
              security       0.00      0.00      0.00       104
              military       1.00      0.01      0.02       178
           child_alone       0.00      0.00      0.00         0
                 water       0.88      0.21      0.34       308
                  food       0.85      0.55      0.66       579
               shelter       0.86      0.30      0.45       452
              clothing       0.67      0.02      0.05        81
                 money       1.00      

/Users/jimeng/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jimeng/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jimeng/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jimeng/opt/anacon

In [13]:
for i in range(len(y.columns)):
    labels = np.unique(y_test.iloc[:,i])
    print(y.columns[i])
    print(classification_report(y_test.iloc[:, i].values, y_pred[:, i]))

related
              precision    recall  f1-score   support

           0       0.67      0.33      0.44      1214
           1       0.82      0.95      0.88      3992

    accuracy                           0.81      5206
   macro avg       0.75      0.64      0.66      5206
weighted avg       0.79      0.81      0.78      5206

request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      4293
           1       0.81      0.48      0.60       913

    accuracy                           0.89      5206
   macro avg       0.85      0.73      0.77      5206
weighted avg       0.88      0.89      0.88      5206

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5182
           1       0.00      0.00      0.00        24

    accuracy                           1.00      5206
   macro avg       0.50      0.50      0.50      5206
weighted avg       0.99      1.00      0.99      520

### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x1a23081170>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          ccp_alpha=0.0,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_depth=None,
                 

In [16]:
parameters = {
        'clf__estimator__max_depth' : [4,6,8],
        'clf__estimator__max_features': ['auto', 'sqrt', 'log2'],
        'clf__estimator__n_estimators': [50, 100, 200]
             }

cv = GridSearchCV(pipeline, param_grid=parameters, cv=3, n_jobs = 4)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
import time 
start_time = time.time()
cv.fit(X_sample_train, y_sample_train)
print ('Model training time is %s minutes' % ((time.time() - start_time)/60))

In [ ]:
y_pred = cv.predict(X_test)

In [ ]:
print(classification_report(y_test.values, y_pred, target_names=y.keys()))

In [ ]:
for i in range(len(y.columns)):
    labels = np.unique(y_test.iloc[:,i])
    print(y.columns[i])
    print(classification_report(y_test.iloc[:, i].values, y_pred[:, i]))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
import pickle
filename = 'model.pkl'
pickle.dump(pipeline, open(filename, 'wb'))


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.